# Dog Breed Prediction

Build, Train and test a convolutional neural network capable of identifying the vreed of a dog in a supplied image. (Supervised ML - Multi class classification problem)

this model can be used by NGOs working on saving anumals and for educational purposes.

## Steps 

1. Load data form Kaggle
2. Load labels CSV for labels that contain image ID and Breed
3. Checking the breed count
4. One-hot encoding on labels data prediction column
5. Load the images, Convert them to an Array and normalize them
6. Check the shape and size of x and y data
7. Bbuild the network model architecture
8. Split the data an dfit in into the model and create an accuaracy plot
9. Evaluate the model for accuracy score
10. Using the model for prediction

In [1]:
## need to install pip install kaggle kaggle-api

In [2]:
import os
import opendatasets as od
from zipfile import ZipFile
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
os.chmod(os.path.expanduser('C:\\Users\\anfes\\My Drive\\Projects\\Data_Projects\\Dog_Breed\\.kaggle\\kaggle.json'), 0o600)  # Nota: 0o600 representa los permisos en octal

# Download the Dataset 

Before downloading it you must download the kaggle API credentials from your profile in kaggle and upload it into a .kaggle directory

In [3]:
# Importa las librerías necesarias
from kaggle.api.kaggle_api_extended import KaggleApi

# Instancia la API de Kaggle
api = KaggleApi()

# Autentica la API utilizando tu archivo de configuración
api.authenticate()

# Descarga el dataset utilizando el comando "dataset_download_files"
dataset_name = "catherinehorng/dogbreedidfromcomp"
destination_path = "dog_dataset"  # Cambia esto a la ruta deseada
api.dataset_download_files(dataset_name, path=destination_path, unzip=True)

#od.download("https://www.kaggle.com/datasets/catherinehorng/dogbreedidfromcomp")
#!kaggle datasets download catherinehorng/dogbreedidfromcomp -p C:\\Users\\anfes\\My Drive\\Projects\\Data_Projects\\Dog_Breed\\dog_dataset
#%cd ..

# Installing libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm   # is a library that provides a visual progress bar for loops and long operations.
from keras.preprocessing import image   # contains utilities for loading and preprocessing images before feeding them into deep learning models.
from sklearn.preprocessing import label_binarize    # is used to convert categorical labels into binary label format.
from sklearn.model_selection import train_test_split    # allows you to split datasets into training and testing subsets
from keras.models import Sequential # is a way to create sequential neural network models layer by layer.
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D # contains the layers you can use to build your model, such as Dense (fully connected dense layer), Dropout (regularization layer), Flatten (flatten data), Conv2D (2D convolutional layer), and MaxPool2D (pooling layer).
from keras.optimizers import Adam   #  implements the Adam optimization algorithm, which is used to adjust model parameters during the training process.

## Reading Labels...